In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
drive_path = '/data/allen-brain-observatory/visual-coding-2p'
manifest_file = os.path.join(drive_path,'manifest.json')

boc = BrainObservatoryCache(manifest_file=manifest_file)

experiment_container_id = 511510640
cell_specimen_id = 517416426
session_id = 502793808

In [22]:
def get_avg_normalized_events(boc, session_id, cell_specimen_id, temporal_frequency=2.0):
    ''' generate normalized average event for each grating orientation
    '''
    data_set = boc.get_ophys_experiment_data(session_id)
    
    events = boc.get_ophys_experiment_events(ophys_experiment_id=session_id)
    
    stim_table = data_set.get_stimulus_table('drifting_gratings')
    
    #Calculate the mean event for each grating presentation in this stimulus
    rows = list()
    for i in range(len(stim_table)):
        new_row = {
            'orientation': stim_table.orientation[i],
            'temporal_frequency': stim_table.temporal_frequency[i],
            'mean_event': events[0,:][stim_table.start[i]:stim_table.end[i]].mean()
        }
        rows.append(new_row)

    cell_response = pd.DataFrame.from_dict(rows)
    tf2_response = cell_response[cell_response.temporal_frequency==temporal_frequency]
    
    mean_event_ori = tf2_response.groupby('orientation').mean()
    
    max_response = mean_event_ori['mean_event'].max()
    
    return mean_event_ori['mean_event']/max_response

In [23]:
get_avg_normalized_events(boc, session_id, cell_specimen_id)

orientation
0.0      0.082340
45.0     0.013921
90.0     0.031474
135.0    0.012731
180.0    0.069807
225.0    0.022283
270.0    0.066667
315.0    0.006671
Name: max_event, dtype: float64